In [1]:
import numpy as np
import pescador
import logging
import os

import matplotlib
import matplotlib.pyplot as plt
import numpy as np

import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.cuda.amp import autocast
from torch.distributions.gamma import Gamma
from torch.distributions.normal import Normal

from IPython.display import display, clear_output

import math

import gc
import sys

from datetime import datetime

from functools import reduce
from torch.distributions.categorical import Categorical
from torch.distributions.one_hot_categorical import OneHotCategorical

In [2]:
LOGGER = logging.getLogger('gbsd')
LOGGER.setLevel(logging.DEBUG)

In [3]:
torch.set_printoptions(sci_mode=False)
np.set_printoptions(suppress=True)

In [4]:
matplotlib.use('Agg')

In [5]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
    
cpu = torch.device('cpu')

In [6]:
CMD_VOLENVPER = 0
CMD_DUTYLL = 1
CMD_MSB = 2
CMD_LSB = 3
CMD_COUNT = 4

TIME_OFFSET = 0
CH_OFFSET = 1
CMD_OFFSET = 2
PARAM1_OFFSET = 3
PARAM2_OFFSET = 4
PARAM3_OFFSET = 5
SIZE_OF_INPUT_FIELDS = 6

MAX_WINDOW_SIZE = 2 * 1024

M_CYCLES_PER_SECOND = 4194304.
NORMALIZE_TIME_BY = M_CYCLES_PER_SECOND * 10.

def fresh_input(command, channel, time):
    newd = np.zeros(shape=SIZE_OF_INPUT_FIELDS, dtype=int)
    newd[TIME_OFFSET] = time
    newd[CH_OFFSET] = channel
    newd[CMD_OFFSET] = command
    return newd

def parse_bool(v):
    if v == "true":
        return 1
    elif v == "false":
        return 0
    else:
        return int(v)

def command_of_parts(command, channel, parts, time):
    inp = fresh_input(command, channel, time)
    
    if command == CMD_DUTYLL:
        inp[PARAM1_OFFSET] = int(parts[3])
        inp[PARAM2_OFFSET] = int(parts[4])
    elif command == CMD_VOLENVPER:
        inp[PARAM1_OFFSET] = int(parts[3])
        inp[PARAM2_OFFSET] = parse_bool(parts[4])
        inp[PARAM3_OFFSET] = int(parts[4])
    elif command == CMD_LSB:
        inp[PARAM1_OFFSET] = int(parts[3])
        inp[PARAM2_OFFSET] = 0
        inp[PARAM3_OFFSET] = 0
    elif command == CMD_MSB:
        inp[PARAM1_OFFSET] = int(parts[3])
        inp[PARAM2_OFFSET] = parse_bool(parts[4])
        inp[PARAM3_OFFSET] = parse_bool(parts[5])
    else:
        raise "this should not happen"
    return inp

def int32_as_bytes(ival):
    return np.frombuffer(ival.item().to_bytes(4, byteorder = 'big'), dtype=np.uint8)

def int32_of_bytes(np):
    return int.from_bytes(np, byteorder = 'big')

def int8_as_bytes(ival):
    return np.frombuffer(ival.item().to_bytes(1, byteorder='big'), dtype=np.uint8)

def int8_of_bytes(np):
    return int.from_bytes(np, byteorder = 'big')

def merge_params(data):
    command = data[CMD_OFFSET]
    if command == CMD_DUTYLL:
        return (data[PARAM1_OFFSET] << 6) | data[PARAM2_OFFSET]
    elif command == CMD_VOLENVPER:
        return (data[PARAM1_OFFSET] << 4) | (data[PARAM2_OFFSET] << 3) | data[PARAM3_OFFSET]
    elif command == CMD_LSB:
        return data[PARAM1_OFFSET]
    elif command == CMD_MSB:
        return data[PARAM1_OFFSET]  | (data[PARAM2_OFFSET] << 6) | (data[PARAM3_OFFSET] << 7)
    else:
        raise "this should not happen"
        
def unmerge_params(command, data,v ):
    if command == CMD_DUTYLL:
        data[PARAM1_OFFSET] = v >> 6;
        data[PARAM2_OFFSET] = v & 0b0011_1111
    elif command == CMD_VOLENVPER:
        data[PARAM1_OFFSET] = v >> 4
        data[PARAM2_OFFSET] = (v & 0b0000_1000) >> 3
        data[PARAM3_OFFSET] = (v & 0b0000_0111)
    elif command == CMD_LSB:
        data[PARAM1_OFFSET] = v
    elif command == CMD_MSB:
        data[PARAM1_OFFSET] = v & 0b0011_1111
        data[PARAM2_OFFSET] = (v & 0b0100_0000) >> 6
        data[PARAM3_OFFSET] = (v & 0b1000_0000) >> 7
    else:
        raise Exception("this should not happen")
        
BYTES_PER_ENTRY=7

def command_to_bytes(command):
    new_arr = np.concatenate([
                    int32_as_bytes(command[TIME_OFFSET]),
                    int8_as_bytes(command[CH_OFFSET]),
                    int8_as_bytes(command[CMD_OFFSET]),
                    int8_as_bytes(merge_params(command)),]).flatten()
    return new_arr

def command_of_bytes(byte_arr):
    d = fresh_input(0, 0, 0)
    d[TIME_OFFSET] = int32_of_bytes(byte_arr[0:4])
    print(byte_arr[0:4], d[TIME_OFFSET])
    d[TIME_OFFSET] = min(d[TIME_OFFSET], 400000)
    print("TOFF:", d[TIME_OFFSET])
    print("CH:", byte_arr, byte_arr[4])
    d[CH_OFFSET] = int8_of_bytes(byte_arr[4:5])
    if d[CH_OFFSET] != 1 and d[CH_OFFSET] != 2:
        raise Exception("bad channel prediction")
    d[CMD_OFFSET] = int8_of_bytes(byte_arr[5:6])
    print("NCMD", d[CMD_OFFSET])
    unmerge_params(d[CMD_OFFSET], d, byte_arr[6])
    return d

def print_feature(data, file=sys.stdout):
    command = data[CMD_OFFSET]
    if command == CMD_DUTYLL:
        print(f"CH {data[CH_OFFSET]} DUTYLL {data[PARAM1_OFFSET]} {data[PARAM2_OFFSET]} AT {data[TIME_OFFSET]}", file=file, flush=True)
    elif command == CMD_VOLENVPER:
        print(f"CH {data[CH_OFFSET]} VOLENVPER {data[PARAM1_OFFSET]} {data[PARAM2_OFFSET]} {data[PARAM3_OFFSET]} AT {data[TIME_OFFSET]}", file=file, flush=True)
    elif command == CMD_LSB:
        print(f"CH {data[CH_OFFSET]} FREQLSB {data[PARAM1_OFFSET]} AT {data[TIME_OFFSET]}", file=file, flush=True)
    elif command == CMD_MSB:
        print(f"CH {data[CH_OFFSET]} FREQMSB {data[PARAM1_OFFSET]} {data[PARAM2_OFFSET]} {data[PARAM3_OFFSET]} AT {data[TIME_OFFSET]}", file=file, flush=True)
    else:
        print(f"Bad prediction", file=file, flush=True)

def load_training_data(src):
    data = []
    file = open(src, 'r')
    for line in file:
        parts = line.split()
        if len(parts) > 0 and parts[0] == "CH":
            #print(parts)
            channel = int(parts[1])
            command = parts[2]
            time = int(parts[-1])
            if command == "DUTYLL":
                new_item = command_of_parts(CMD_DUTYLL, channel, parts, time)
            elif command == "VOLENVPER":
                new_item = command_of_parts(CMD_VOLENVPER, channel, parts, time)
            elif command == "FREQLSB":
                new_item = command_of_parts(CMD_LSB, channel, parts, time)
            elif command == "FREQMSB":
                new_item = command_of_parts(CMD_MSB, channel, parts, time)
            else:
                print("Unknown", command)
             # Otherwise unknown   
            data.append(new_item)
    return data

@pescador.streamable
def samples_from_training_data(src, window_size):
    sample_data = None
    try:
        sample_data = load_training_data(src)
    except Exception as e:
        LOGGER.error('Could not load {}: {}'.format(src, str(e)))
        raise StopIteration()

    while True:
        if len(sample_data) < window_size:
            sample = sample_data
        else:
            # Sample a random window from the audio file
            start_idx = np.random.randint(0, len(sample_data) - window_size)
            sample = sample_data[start_idx:(start_idx + window_size)]
            
        sample = np.array([command_to_bytes(x) for x in sample])

        yield sample

def create_batch_generator(paths, window_size):
    streamers = []
    
    for path in paths:
        streamers.append(samples_from_training_data(path, window_size))
    
    mux = pescador.StochasticMux(streamers, n_active=1, rate=1).iterate()
    
    return mux

def training_files(dirp):
    return [
      os.path.join(root, fname)
      for (root, dir_names, file_names) in os.walk(dirp, followlinks=True)
      for fname in file_names
    ]

def create_data_split(paths, window_size=MAX_WINDOW_SIZE):
    train_gen = create_batch_generator(paths, window_size)
    return train_gen

class SampleDataset(torch.utils.data.IterableDataset):
    
    def __init__(self, path, window_size):
        super(SampleDataset).__init__()
        self.loader = create_data_split(training_files(path))
    
    def __iter__(self):
        while True:
             yield next(self.loader).flatten()

In [7]:
class PositionalEncoding(nn.Module):

    def __init__(self, d_model, dropout = 0.1, max_len = 2048):
        super().__init__()
        
        assert(MAX_WINDOW_SIZE <= max_len)
        
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0)]
        return self.dropout(x)

In [8]:
KERNEL_SIZE_SAMPLES=16
KERNEL_SIZE=BYTES_PER_ENTRY * KERNEL_SIZE_SAMPLES

class CausalConv1d(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, dilation=1, **kwargs):
        super(CausalConv1d, self).__init__()
        self.pad = (kernel_size - 1) * dilation
        self.conv = nn.Conv1d(in_channels, out_channels, kernel_size , dilation=dilation, **kwargs)
        
    def forward(self, x):
        #pad here to only add to the left side
        x = F.pad(x, (self.pad, 0))
        return self.conv(x)

class ResidualBlock(nn.Module):
    def __init__(self, input_channels, output_channels, kernel_size, skip_channels, dilation=1):
        super(ResidualBlock, self).__init__()
        
        self.conv_sig = CausalConv1d(input_channels, output_channels, kernel_size, dilation)
        self.sig = nn.Sigmoid()
        
        self.conv_tan = CausalConv1d(input_channels, output_channels, kernel_size, dilation)
        self.tanh = nn.Tanh()
        
        #separate weights for residual and skip channels
        self.conv_r = nn.Conv1d(output_channels, output_channels, 1)
        self.conv_s = nn.Conv1d(output_channels, skip_channels, 1)
        
    def forward(self, x):
        o = self.sig(self.conv_sig(x)) * self.tanh(self.conv_tan(x))
        skip = self.conv_s(o)
        residual = self.conv_r(o)
        return residual, skip

# When using dilations the effective lookback is KERNEL_SIZE^num_layers
class CommandNet(nn.Module):
    def __init__(self, skip_channels=256, num_blocks=4, num_layers=5, num_hidden=256, kernel_size=KERNEL_SIZE): 
        super(CommandNet, self).__init__()

        self.embed = nn.Embedding(skip_channels, skip_channels)
        #self.positional_embedding = PositionalEncoding(skip_channels)
        self.causal_conv = CausalConv1d(skip_channels, num_hidden, kernel_size)
        self.res_stack = nn.ModuleList()

        for b in range(num_blocks):
            for i in range(num_layers):
                self.res_stack.append(ResidualBlock(num_hidden, num_hidden, kernel_size, skip_channels=skip_channels, dilation=2**i))
        
        self.relu1 = nn.ReLU()
        self.conv1 = nn.Conv1d(skip_channels, skip_channels, 1)
        self.relu2 = nn.ReLU()
        self.conv2 = nn.Conv1d(skip_channels, skip_channels, 1)
        
    def forward(self, x):

        o = self.embed(x)
        #o = self.positional_embedding(o)
        o = o.permute(0,2,1)
        
        o = self.causal_conv(o)
        
        skip_vals = []
        
        #run res blocks
        for i, layer in enumerate(self.res_stack):
            o, s = layer(o)
            skip_vals.append(s)
            
        #sum skip values and pass to last portion of network
        o = reduce((lambda a,b: a+b), skip_vals)
        
        o = self.relu1(o)
        o = self.conv1(o)
        o = self.relu2(o)
        o = self.conv2(o)
        
        return o

def load(path):
        
    lr = 0.01
    momentum=0.8
    
    command_generator = CommandNet()
    
    optimizer = optim.SGD(
        command_generator.parameters(),
        lr=lr,
        momentum=momentum
    )
    
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', factor=0.95, min_lr=0.001)
    command_generator = command_generator.to(device)
    
    # This needs to be after to because the optimizer decides what device to send the tensors to based on the
    # device of the model.
    if path != None:
        command_generator.load_state_dict(torch.load(path + ".model"))
        optimizer.load_state_dict(torch.load(path + ".optimizer"))
        #scheduler = torch.load(path + ".scheduler")

    return command_generator, optimizer, scheduler

In [9]:
EPOCHS = 500000
ROUND_SZ = 100

print("Collecting training data")
loader = torch.utils.data.DataLoader(SampleDataset("../../training_data/",window_size=MAX_WINDOW_SIZE))
print("Collected")

def train(path):
    
    command_generator, optimizer, scheduler = load(path)

    criterion = nn.CrossEntropyLoss()
    running_loss = torch.zeros(1, device=device)

    def step():
        
        print("Starting batch")
        running_loss.zero_()
        
        for i in range(ROUND_SZ):
            
            if i % (ROUND_SZ / 10) == 0:
                print("Batch completion:", (float(i) / float(ROUND_SZ)) * 100., "%")
            
            seq = next(iter(loader)).long().to(device)
            inputs = seq[:,:-1]
            labels = seq[:,1:]
    
            optimizer.zero_grad()

            with autocast():
                outputs = command_generator(inputs)
                
                # Backprop only on the datapoints that had at least half a k kernel
                #backprop_l = int(min(KERNEL_SIZE / 2, len(seq) / 2))
                #backprop_inputs = outputs[:,:,backprop_l:]
                #backprop_outputs = labels[:,backprop_l:]
                
                loss = criterion(outputs, labels)

            loss.backward()
            optimizer.step()
            #print(loss.detach().item())
            running_loss.add_(loss.detach())
            #print(running_loss)
            
            seq = seq.detach().to(cpu)
            del inputs
            del labels
            del seq

        result = running_loss / ROUND_SZ
        return result
    
    def save(name):
        torch.save(command_generator.state_dict(), "./" + name + ".model")
        torch.save(optimizer.state_dict(), "./" + name + ".optimizer")
        
        # Saving the scheduler seems to break stuff
        #torch.save(scheduler, "./" + name + ".scheduler")

    for i in range(0, EPOCHS):
        loss = step()
        scheduler.step(loss)

        print("Loss:", loss.item())
        print("LR:", optimizer.param_groups[0]['lr'])
        
        print("Saving checkpoint")
        
        # Timestamp every 10th epoch to test fits later
        if i % 10 == 0:
            save(str(int(datetime.now().timestamp())))

        save("./last.checkpoint")
        print("Saved checkpoint")
    
    return command_generator.eval()

#train(None)
#train("./last.checkpoint")

Collected


In [ ]:
# %%capture cap --no-stderr

print("Collecting training data")
train_gen = create_data_split(training_files("../../out_of_sample/"))
print("Collected")

command_generator, _, _ = load("./last.checkpoint")
command_generator = command_generator.eval()

seed = next(train_gen).flatten()

def max_of(v, begin, end):
    return begin + np.argmax(v[begin:end])

with open('seed.txt', 'w') as f:
    for i in range(0, len(seed), BYTES_PER_ENTRY):
        print("Seed value :", i, seed.shape)
        cmd = command_of_bytes(seed[i:i+BYTES_PER_ENTRY])
        print_feature(cmd, file=f)
        
class MovingWindow():
    
    def __init__(self, seed):
        # Pre-allocate 4x the seed
        self.seq = torch.cat((torch.Tensor(seed).long(), torch.zeros(len(seed) * 4).long())).to(device)
        self.start = 0
        self.len = len(seed)
        
    def append(self, item):
        self.seq[self.start + self.len] = item
        self.start += 1

    def window(self):
        # Slice the current window
        return self.seq[self.start:self.start+self.len]
    
window = MovingWindow(seed)

with open('output.txt', 'w') as f:
    
    for i in range(BYTES_PER_ENTRY * 10000):
        seq = window.window().unsqueeze(0)
        pred = command_generator(seq).detach().to(cpu).permute(0,2,1).squeeze(0)[-1]
        pred = Categorical(logits=pred).sample()
        window.append(pred)
    
        if (i + 1) % BYTES_PER_ENTRY == 0:
            try:
                last_sample = window.window()[-BYTES_PER_ENTRY:].detach().cpu().numpy().astype(np.uint8)
                last_sample = command_of_bytes(last_sample)
                print_feature(last_sample, file=f)
            except BaseException as err:
                print("pred was not valid because:", err)

    del pred

Collected
Seed value : 0 (12656,)
[ 0  0  0 20] 20
TOFF: 20
CH: [ 0  0  0 20  1  1  0] 1
NCMD 1
Seed value : 7 (12656,)
[ 0  0  0 20] 20
TOFF: 20
CH: [ 0  0  0 20  1  0  0] 1
NCMD 0
Seed value : 14 (12656,)
[ 0  0  0 20] 20
TOFF: 20
CH: [ 0  0  0 20  2  1  0] 2
NCMD 1
Seed value : 21 (12656,)
[ 0  0  0 20] 20
TOFF: 20
CH: [ 0  0  0 20  2  0  0] 2
NCMD 0
Seed value : 28 (12656,)
[ 0  0  0 60] 60
TOFF: 60
CH: [  0   0   0  60   1   1 128] 1
NCMD 1
Seed value : 35 (12656,)
[ 0  0  0 76] 76
TOFF: 76
CH: [  0   0   0  76   1   0 176] 1
NCMD 0
Seed value : 42 (12656,)
[  0   0   0 248] 248
TOFF: 248
CH: [  0   0   0 248   1   3  68] 1
NCMD 3
Seed value : 49 (12656,)
[ 0  0  0 28] 28
TOFF: 28
CH: [  0   0   0  28   1   2 135] 1
NCMD 2
Seed value : 56 (12656,)
[  0   0   0 204] 204
TOFF: 204
CH: [  0   0   0 204   2   1 128] 2
NCMD 1
Seed value : 63 (12656,)
[ 0  0  0 76] 76
TOFF: 76
CH: [  0   0   0  76   2   0 240] 2
NCMD 0
Seed value : 70 (12656,)
[  0   0   0 248] 248
TOFF: 248
CH: [  0   

Seed value : 1736 (12656,)
[ 0  0  0 28] 28
TOFF: 28
CH: [  0   0   0  28   1   2 134] 1
NCMD 2
Seed value : 1743 (12656,)
[  0   0   1 136] 392
TOFF: 392
CH: [  0   0   1 136   2   3  44] 2
NCMD 3
Seed value : 1750 (12656,)
[ 0  0  0 28] 28
TOFF: 28
CH: [  0   0   0  28   2   2 128] 2
NCMD 2
Seed value : 1757 (12656,)
[  0  16 190 228] 1097444
TOFF: 400000
CH: [  0  16 190 228   1   3 231] 1
NCMD 3
Seed value : 1764 (12656,)
[ 0  0  0 28] 28
TOFF: 28
CH: [  0   0   0  28   1   2 134] 1
NCMD 2
Seed value : 1771 (12656,)
[  0   8  96 124] 548988
TOFF: 400000
CH: [  0   8  96 124   1   3 196] 1
NCMD 3
Seed value : 1778 (12656,)
[ 0  0  0 28] 28
TOFF: 28
CH: [  0   0   0  28   1   2 134] 1
NCMD 2
Seed value : 1785 (12656,)
[  0  16 192 252] 1097980
TOFF: 400000
CH: [  0  16 192 252   1   3 231] 1
NCMD 3
Seed value : 1792 (12656,)
[ 0  0  0 28] 28
TOFF: 28
CH: [  0   0   0  28   1   2 134] 1
NCMD 2
Seed value : 1799 (12656,)
[  0   8  96 124] 548988
TOFF: 400000
CH: [  0   8  96 124   1   

Seed value : 3528 (12656,)
[ 0  0  0 28] 28
TOFF: 28
CH: [  0   0   0  28   2   2 131] 2
NCMD 2
Seed value : 3535 (12656,)
[ 0  8 96 32] 548896
TOFF: 400000
CH: [  0   8  96  32   1   3 206] 1
NCMD 3
Seed value : 3542 (12656,)
[ 0  0  0 28] 28
TOFF: 28
CH: [  0   0   0  28   1   2 133] 1
NCMD 2
Seed value : 3549 (12656,)
[  0   0   1 136] 392
TOFF: 392
CH: [  0   0   1 136   2   3 155] 2
NCMD 3
Seed value : 3556 (12656,)
[ 0  0  0 28] 28
TOFF: 28
CH: [  0   0   0  28   2   2 131] 2
NCMD 2
Seed value : 3563 (12656,)
[  0  16 192   0] 1097728
TOFF: 400000
CH: [  0  16 192   0   2   3 155] 2
NCMD 3
Seed value : 3570 (12656,)
[ 0  0  0 28] 28
TOFF: 28
CH: [  0   0   0  28   2   2 131] 2
NCMD 2
Seed value : 3577 (12656,)
[  0   8  96 132] 548996
TOFF: 400000
CH: [  0   8  96 132   2   3 155] 2
NCMD 3
Seed value : 3584 (12656,)
[ 0  0  0 28] 28
TOFF: 28
CH: [  0   0   0  28   2   2 131] 2
NCMD 2
Seed value : 3591 (12656,)
[  0   8  96 132] 548996
TOFF: 400000
CH: [  0   8  96 132   2   3 155

Seed value : 5341 (12656,)
[ 0  0  0 28] 28
TOFF: 28
CH: [  0   0   0  28   2   2 131] 2
NCMD 2
Seed value : 5348 (12656,)
[ 0  8 95 28] 548636
TOFF: 400000
CH: [  0   8  95  28   2   3 181] 2
NCMD 3
Seed value : 5355 (12656,)
[ 0  0  0 28] 28
TOFF: 28
CH: [  0   0   0  28   2   2 132] 2
NCMD 2
Seed value : 5362 (12656,)
[  0   8  96 108] 548972
TOFF: 400000
CH: [  0   8  96 108   2   3  17] 2
NCMD 3
Seed value : 5369 (12656,)
[ 0  0  0 28] 28
TOFF: 28
CH: [  0   0   0  28   2   2 133] 2
NCMD 2
Seed value : 5376 (12656,)
[  0  12 143  68] 823108
TOFF: 400000
CH: [  0  12 143  68   1   0 176] 1
NCMD 0
Seed value : 5383 (12656,)
[  0   0   0 248] 248
TOFF: 248
CH: [  0   0   0 248   1   3   6] 1
NCMD 3
Seed value : 5390 (12656,)
[ 0  0  0 28] 28
TOFF: 28
CH: [  0   0   0  28   1   2 135] 1
NCMD 2
Seed value : 5397 (12656,)
[  0   0   1 136] 392
TOFF: 392
CH: [  0   0   1 136   2   3  78] 2
NCMD 3
Seed value : 5404 (12656,)
[ 0  0  0 28] 28
TOFF: 28
CH: [  0   0   0  28   2   2 132] 2
NCM

Seed value : 7168 (12656,)
[ 0  0  0 28] 28
TOFF: 28
CH: [  0   0   0  28   1   2 135] 1
NCMD 2
Seed value : 7175 (12656,)
[  0   0   1 136] 392
TOFF: 392
CH: [  0   0   1 136   2   3  18] 2
NCMD 3
Seed value : 7182 (12656,)
[ 0  0  0 28] 28
TOFF: 28
CH: [  0   0   0  28   2   2 131] 2
NCMD 2
Seed value : 7189 (12656,)
[  0  16 190 132] 1097348
TOFF: 400000
CH: [  0  16 190 132   1   0 160] 1
NCMD 0
Seed value : 7196 (12656,)
[  0   0   0 248] 248
TOFF: 248
CH: [  0   0   0 248   1   3 116] 1
NCMD 3
Seed value : 7203 (12656,)
[ 0  0  0 28] 28
TOFF: 28
CH: [  0   0   0  28   1   2 135] 1
NCMD 2
Seed value : 7210 (12656,)
[  0  16 192  36] 1097764
TOFF: 400000
CH: [  0  16 192  36   1   0 144] 1
NCMD 0
Seed value : 7217 (12656,)
[  0   0   0 248] 248
TOFF: 248
CH: [  0   0   0 248   1   3 116] 1
NCMD 3
Seed value : 7224 (12656,)
[ 0  0  0 28] 28
TOFF: 28
CH: [  0   0   0  28   1   2 135] 1
NCMD 2
Seed value : 7231 (12656,)
[  0  16 192  28] 1097756
TOFF: 400000
CH: [  0  16 192  28   1  

...

[ 0  1  6 56] 67128
TOFF: 67128
CH: [  0   1   6  56   1   3 229] 1
NCMD 3
[ 0  0  0 28] 28
TOFF: 28
CH: [ 0  0  0 28  1  2  1] 1
NCMD 2
[ 0  1  6 56] 67128
TOFF: 67128
CH: [  0   1   6  56   1   3 137] 1
NCMD 3
[ 0  0  0 28] 28
TOFF: 28
CH: [ 0  0  0 28  1  2  5] 1
NCMD 2
[  0   1   6 152] 67224
TOFF: 67224
CH: [  0   1   6 152   1   3 229] 1
NCMD 3
[ 0  0  0 28] 28
TOFF: 28
CH: [ 0  0  0 28  1  2  1] 1
NCMD 2
[  0   1   6 232] 67304
TOFF: 67304
CH: [  0   1   6 232   1   3 137] 1
NCMD 3
[ 0  0  0 28] 28
TOFF: 28
CH: [ 0  0  0 28  1  2  5] 1
NCMD 2
[  0   1   6 228] 67300
TOFF: 67300
CH: [  0   1   6 228   1   3 229] 1
NCMD 3
[ 0  0  0 28] 28
TOFF: 28
CH: [ 0  0  0 28  1  2  4] 1
NCMD 2
[  0   1   6 232] 67304
TOFF: 67304
CH: [  0   1   6 232   1   3 137] 1
NCMD 3
[ 0  0  0 28] 28
TOFF: 28
CH: [ 0  0  0 28  1  2  5] 1
NCMD 2
[  0   1   6 168] 67240
TOFF: 67240
CH: [  0   1   6 168   1   3 229] 1
NCMD 3
[ 0  0  0 28] 28
TOFF: 28
CH: [ 0  0  0 28  1  2  4] 1
NCMD 2
[ 0  1  6 60] 67132
T

[  0   0   1 184] 440
TOFF: 440
CH: [  0   0   1 184   2   3  99] 2
NCMD 3
[ 0  0  0 28] 28
TOFF: 28
CH: [ 0  0  0 28  2  2  7] 2
NCMD 2
[ 0  1  4 84] 66644
TOFF: 66644
CH: [ 0  1  4 84  1  3 59] 1
NCMD 3
[ 0  0  0 28] 28
TOFF: 28
CH: [ 0  0  0 28  1  2  5] 1
NCMD 2
[  0   0   1 184] 440
TOFF: 440
CH: [  0   0   1 184   2   3  99] 2
NCMD 3
[ 0  0  0 28] 28
TOFF: 28
CH: [ 0  0  0 28  2  2  7] 2
NCMD 2
[ 0  1  4 84] 66644
TOFF: 66644
CH: [ 0  1  4 84  1  3 59] 1
NCMD 3
[ 0  0  0 28] 28
TOFF: 28
CH: [ 0  0  0 28  1  2  5] 1
NCMD 2
[  0   0   1 184] 440
TOFF: 440
CH: [  0   0   1 184   2   3  99] 2
NCMD 3
[ 0  0  0 28] 28
TOFF: 28
CH: [ 0  0  0 28  2  2  7] 2
NCMD 2
[ 0  1  4 84] 66644
TOFF: 66644
CH: [ 0  1  4 84  1  3 59] 1
NCMD 3
[ 0  0  0 28] 28
TOFF: 28
CH: [ 0  0  0 28  1  2  5] 1
NCMD 2
[  0   0   1 184] 440
TOFF: 440
CH: [  0   0   1 184   2   3  98] 2
NCMD 3
[ 0  0  0 28] 28
TOFF: 28
CH: [ 0  0  0 28  2  2  7] 2
NCMD 2
[ 0  1  4 84] 66644
TOFF: 66644
CH: [ 0  1  4 84  1  3 59] 1
N

[ 0  1  6 32] 67104
TOFF: 67104
CH: [  0   1   6  32   2   3 123] 2
NCMD 3
[ 0  0  0 28] 28
TOFF: 28
CH: [ 0  0  0 28  2  2  7] 2
NCMD 2
[ 0  1  6 32] 67104
TOFF: 67104
CH: [  0   1   6  32   2   3 123] 2
NCMD 3
[ 0  0  0 28] 28
TOFF: 28
CH: [ 0  0  0 28  2  2  7] 2
NCMD 2
[ 0  1  5 28] 66844
TOFF: 66844
CH: [  0   1   5  28   1   0 240] 1
NCMD 0
[ 0  0  0 36] 36
TOFF: 36
CH: [  0   0   0  36   1   1 128] 1
NCMD 1
[  0   0   0 168] 168
TOFF: 168
CH: [  0   0   0 168   1   3 229] 1
NCMD 3
[ 0  0  0 60] 60
TOFF: 60
CH: [  0   0   0  60   1   2 132] 1
NCMD 2
[ 0  0  1 84] 340
TOFF: 340
CH: [  0   0   1  84   2   0 208] 2
NCMD 0
[ 0  0  0 52] 52
TOFF: 52
CH: [  0   0   0  52   2   2 135] 2
NCMD 2
[  0   0   0 168] 168
TOFF: 168
CH: [  0   0   0 168   2   3 130] 2
NCMD 3
[ 0  0  0 60] 60
TOFF: 60
CH: [  0   0   0  60   2   2 135] 2
NCMD 2
[ 0  1  3 36] 66340
TOFF: 66340
CH: [  0   1   3  36   1   3 131] 1
NCMD 3
[ 0  0  0 28] 28
TOFF: 28
CH: [ 0  0  0 28  1  2  4] 1
NCMD 2
[  0   0   1 184]

[ 0  0  0 36] 36
TOFF: 36
CH: [  0   0   0  36   1   1 128] 1
NCMD 1
[  0   0   0 168] 168
TOFF: 168
CH: [  0   0   0 168   1   3  96] 1
NCMD 3
[ 0  0  0 60] 60
TOFF: 60
CH: [  0   0   0  60   1   2 132] 1
NCMD 2
[ 0  0  1 84] 340
TOFF: 340
CH: [  0   0   1  84   2   0 208] 2
NCMD 0
[ 0  0  0 52] 52
TOFF: 52
CH: [  0   0   0  52   2   1 128] 2
NCMD 1
[  0   0   0 168] 168
TOFF: 168
CH: [  0   0   0 168   2   3  88] 2
NCMD 3
[ 0  0  0 60] 60
TOFF: 60
CH: [  0   0   0  60   2   2 135] 2
NCMD 2
[ 0  1  3 36] 66340
TOFF: 66340
CH: [ 0  1  3 36  1  3 96] 1
NCMD 3
[ 0  0  0 28] 28
TOFF: 28
CH: [ 0  0  0 28  1  2  4] 1
NCMD 2
[  0   0   1 184] 440
TOFF: 440
CH: [  0   0   1 184   2   3  88] 2
NCMD 3
[ 0  0  0 28] 28
TOFF: 28
CH: [ 0  0  0 28  2  2  7] 2
NCMD 2
[ 0  1  4 84] 66644
TOFF: 66644
CH: [ 0  1  4 84  1  3 96] 1
NCMD 3
[ 0  0  0 28] 28
TOFF: 28
CH: [ 0  0  0 28  1  2  4] 1
NCMD 2
[  0   0   1 184] 440
TOFF: 440
CH: [  0   0   1 184   2   3  89] 2
NCMD 3
[ 0  0  0 28] 28
TOFF: 28
CH: [

[ 0  0  0 28] 28
TOFF: 28
CH: [ 0  0  0 28  1  2  5] 1
NCMD 2
[  0   0   1 184] 440
TOFF: 440
CH: [  0   0   1 184   2   3  58] 2
NCMD 3
[ 0  0  0 28] 28
TOFF: 28
CH: [ 0  0  0 28  2  2  7] 2
NCMD 2
[ 0  1  4 84] 66644
TOFF: 66644
CH: [ 0  1  4 84  1  3 17] 1
NCMD 3
[ 0  0  0 28] 28
TOFF: 28
CH: [ 0  0  0 28  1  2  5] 1
NCMD 2
[  0   0   1 184] 440
TOFF: 440
CH: [  0   0   1 184   2   3  57] 2
NCMD 3
[ 0  0  0 28] 28
TOFF: 28
CH: [ 0  0  0 28  2  2  7] 2
NCMD 2
[ 0  1  4 84] 66644
TOFF: 66644
CH: [ 0  1  4 84  1  3 17] 1
NCMD 3
[ 0  0  0 28] 28
TOFF: 28
CH: [ 0  0  0 28  1  2  5] 1
NCMD 2
[  0   0   1 184] 440
TOFF: 440
CH: [  0   0   1 184   2   3  57] 2
NCMD 3
[ 0  0  0 28] 28
TOFF: 28
CH: [ 0  0  0 28  2  2  7] 2
NCMD 2
[ 0  1  4 84] 66644
TOFF: 66644
CH: [ 0  1  4 84  1  3 17] 1
NCMD 3
[ 0  0  0 28] 28
TOFF: 28
CH: [ 0  0  0 28  1  2  5] 1
NCMD 2
[  0   0   1 184] 440
TOFF: 440
CH: [  0   0   1 184   2   3  57] 2
NCMD 3
[ 0  0  0 28] 28
TOFF: 28
CH: [ 0  0  0 28  2  2  7] 2
NCMD 2
